# EVENT PLOTS (GUI version)

A NB template to produce event plots (GUI version)

In [ ]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
%gui qt
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
import time
import tables as tb
import numpy as np

In [ ]:
from invisible_cities.database import load_db

In [ ]:
import invisible_cities.reco.tbl_functions as tbl
import invisible_cities.reco.wfm_functions as wfm
import invisible_cities.sierpe.blr as blr
import invisible_cities.reco.peak_functions_c as cpf
from invisible_cities.core.mpl_functions import set_plot_labels
from invisible_cities.reco.params import S12Params, ThresholdParams
from   invisible_cities.core.system_of_units_c import units

In [ ]:
from gui_functions import fplot_pmt_waveforms, fplot_pmt_signals_vs_time_mus, fplot_signal_vs_time_mus,\
                           fplot_s12, hist_1d

In [ ]:
from event_pmaps import EventPmaps, print_s12, print_s2si

In [ ]:
run_number = 3389

In [ ]:
DataPMT = load_db.DataPMT(run_number)

In [ ]:
DataPMT

In [ ]:
from PyQt5 import QtCore, QtWidgets, uic
from matplotlib.backends.backend_qt5agg import (
    FigureCanvasQTAgg as FigureCanvas,
    NavigationToolbar2QT as NavigationToolbar)
from matplotlib.figure import Figure

from PyQt5.uic import loadUiType


qtCreatorFile = "event_maps.ui" # Enter file here.
Ui_MainWindow, QMainWindow = loadUiType(qtCreatorFile)

In [ ]:
class QtGui(QMainWindow, Ui_MainWindow):
    def __init__(self, ):
        super(QtGui, self).__init__()
        self.setupUi(self)
        
        self.fig_dict = {}
        self.mplfigs.itemClicked.connect(self.change_figure)

        fig = Figure()
        self.add_mpl(fig)

    def add_figure(self, name, fig):
        test_key = self.fig_dict.pop(name, None)
        self.fig_dict[name] = fig
        if not test_key: # key not in dict
            self.mplfigs.addItem(name)

    def change_figure(self, item):
        text = item.text()
        self.rm_mpl()
        self.add_mpl(self.fig_dict[text])

    def add_mpl(self, fig):
        self.canvas = FigureCanvas(fig)
        self.mplvl.addWidget(self.canvas)
        self.canvas.draw()
        self.toolbar = NavigationToolbar(self.canvas,
                self.mplwindow, coordinates=True)
        self.mplvl.addWidget(self.toolbar)

    def rm_mpl(self,):
        self.mplvl.removeWidget(self.canvas)
        self.canvas.close()
        self.mplvl.removeWidget(self.toolbar)
        self.toolbar.close()



In [ ]:
def scan_events(epm, gui, pmtrwf, sipmrwf, 
                event_list=list(range(1)), 
                event_plot_list=list(range(1)), 
                printout=10):
    """Loop over events"""
    
    for event in event_list:
        if(event%printout == 0):
            print('processing event number {}'.format(event))
        epm.calibrated_pmt_and_csum(event, pmtrwf)
        epm.calibrated_sipm(event, sipmrwf)
        epm.find_s1()
        epm.find_s2()
        if (len(epm.S2) > 0):
            epm.find_s2si()
        
        if event in event_plot_list:
            #gui.add_figure('RWF-event-{}'.format(event), 
            #                plot_pmt_waveforms(epm.RWF, zoom=False, window_size=10000))
            gui.add_figure('CWF_vs_time_mus-event-{}'.format(event), 
                            fplot_pmt_signals_vs_time_mus(epm.CWF,
                                                         epm.P.pmt_active,
                                                         t_min      =    0,
                                                         t_max      = 1300,
                                                         signal_min =    -5,
                                                         signal_max =  200))
            gui.add_figure('Calibrated_SUM-event-{}'.format(event), 
                            fplot_signal_vs_time_mus(epm.csum, t_min=0, t_max=1300, signal_min=-5, signal_max=100))
            if (len(epm.S1) > 0):
                gui.add_figure('S1-event-{}'.format(event), fplot_s12(epm.S1))
            if (len(epm.S2) > 0):
                gui.add_figure('S2-event-{}'.format(event), fplot_s12(epm.S2))
                

In [ ]:
RWF_file = os.path.join(os.environ['IC_DATA'],'LSC/wvfms/3389/dst_waves.gdcsnext.000_3389.root.h5')


In [ ]:
h5rwf = tb.open_file(RWF_file,'r')

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

In [ ]:
s1par  = S12Params(tmin=0*units.mus, tmax=640*units.mus, lmin=5, lmax=20, stride=4, rebin=False)
s2par = S12Params(tmin=640*units.mus, tmax=800*units.mus, stride=40, lmin=80, lmax=20000, rebin=True)
thr    = ThresholdParams(thr_s1=0.5 * units.pes,  thr_s2=1 *units.pes,
                        thr_MAU = 3 * units.adc, thr_sipm = 3.5 * units.pes,
                        thr_SIPM = 30 * units.adc)

In [ ]:
gui=QtGui()
gui.show()

In [ ]:
epm = EventPmaps(run_number, s1par, s2par, thr, verbose=True)

In [ ]:
event=0
epm.calibrated_pmt_and_csum(event, pmtrwf)

In [ ]:
gui.add_figure('CWF-event-{}'.format(event), 
                fplot_pmt_signals_vs_time_mus(epm.CWF,
                                             epm.P.pmt_active,
                                             t_min      =    0,
                                             t_max      = 1300,
                                             signal_min =    -5,
                                             signal_max =  250))

In [ ]:
gui.add_figure('PMTs-event-{}'.format(event), 
                fplot_pmt_signals_vs_time_mus(epm.CAL_PMT,
                                             epm.P.pmt_active,
                                             t_min      =    0,
                                             t_max      = 1300,
                                             signal_min =    -5,
                                             signal_max =  10))

In [ ]:
n_events = 160
sample_plot = 10
t0 = time.time()
scan_events(epm, gui, pmtrwf, sipmrwf, 
            event_list=list(range(n_events)), 
            event_plot_list=list(range(0,n_events,sample_plot)),
           printout=10)
t1 = time.time()        
dt = t1 - t0
print("run {} events in {} s event/s = {}".format(n_events, dt, dt/n_events))

In [ ]:
epm.s1f

In [ ]:
gui.add_figure('s1 peaks ',hist_1d(epm.s1f.epeak(), xlo=0, xhi=max(epm.s1f.peak()) + 1))
gui.add_figure('s1 width in ns',hist_1d(epm.s1f.width(), xlo=0, xhi=500))
gui.add_figure('s1 t in mus',hist_1d(epm.s1f.tpeak()/units.mus, xlo=0, xhi=650))
gui.add_figure('s1 etot in pes',hist_1d(epm.s1f.etot(), xlo=0, xhi=max(epm.s1f.etot())+10))
gui.add_figure('s1 etot in pes (2)',hist_1d(epm.s1f.etot(), xlo=0, xhi=50))
gui.add_figure('s1 emax in pes',hist_1d(epm.s1f.emax(), xlo=0, xhi=max(epm.s1f.emax())+10))

In [ ]:
gui.add_figure('s2 peaks ',hist_1d(epm.s2f.peak(), xlo=0, xhi=max(epm.s2f.peak()) + 1))
gui.add_figure('s2 width in mua',hist_1d(epm.s2f.width()/units.mus, xlo=0, xhi=max(epm.s2f.width()/units.mus)+10))
gui.add_figure('s2 t in mus',hist_1d(epm.s2f.tpeak()/units.mus, xlo=650, xhi=670))
gui.add_figure('s2 etot in pes',hist_1d(epm.s2f.etot(), xlo=0, xhi=max(epm.s2f.etot())+100))
gui.add_figure('s2 emax in pes',hist_1d(epm.s2f.emax(), xlo=0, xhi=max(epm.s2f.emax())+100))
gui.add_figure('s2 emax/etot in pes',hist_1d(epm.s2f.emax_over_etot(), xlo=0, xhi=1))